# Example: Kinetic model of isomerization reaction

## Introduction


The problem addressed refers to the parameter estimation of the kinetic model of the pyrolysis reaction of bicyclohexane in the vapor phase. This reaction has as product 1,5-hexadien [1] and is exemplified by Figure 1.

**Figure 1: Bicyclohexane pyrolysis reaction. Font: [1]**
![figura](./Imagens/P.jpg)

The experimental data for this reaction, in a temperature range of 327 °C to 366 °C, are presented in Table 1.                                    
**Table 1: Experimental data. Font: [1]**     
                                               
  Temperature / °C | Reagent pressure / mmHg | Time / min | Reagent fraction / % |       $k$ x $10^{5}$
  :-------------: |  :----------------------: |:-----------:|:---------------------: |:------------------------:
         327.2    |         18.4                        |120              |90.0                  |1.46 
         327.2    |         18.3                        |60.0             |94.9                  |1.43
         338.9    |         18.3                        |60.0             |88.6                  |3.37
             .    |         .                           |.                |.                     |.
             .    |         .                           |.                |.                     |.
             .    |         .                           |.                |.                     |.
         366.3    |         0.45                        |60.0             |46.8                  |21.1  
         366.3    |         0.30                        |30.0             |47.6                  |21.8
         366.3    |         0.25                        |60.0             |49.7                  |19.4
           
           
The reaction kinetic model is given by [1]: 

$y = \exp\left[-\left(K_0\cdot 10^{17}\right)\cdot t \cdot \exp\left(-\frac{E}{T}\right)\right]$,                (1)

on what $y$ is the reagent fraction, $t$ is the time, $T$ is the temperature (K) and $k_o$ and $E$ are the parameters that will be estimated.

The optimization problem to be solved uses the objective function of least squares weighted by the inverse of the variance, according to [2]:

$\min_{ko,E} \sum_{i=1}^{NE} \left(\frac{y^{exp}_i-y_i(k_o,E)}{u^2_{y_i}}\right)$ (2)

subject to (1).

The following symbols will be used to solve this problem in the MT_PEU:

* Symbols of the independent quantities (time and temperature): t, T
* Symbols of the dependent quantities (reagent fraction): y
* Symbols of the parameters: ko, E

## Packages importing

Importing libraries (packages) needed to run the code.

* **MT_PEU**: library that contains the main functionalities of the tool

    * Import the class **EstimacaoNaoLinear**, that will be used in this non-linear estimation example.

* **casadi**: library for symbolic computation

    * only the function **exp** (exponential) will be required to build the model.

In [2]:
from MT_PEU import EstimacaoNaoLinear
from casadi import exp

## Model creation

The model (1) represents the behavior of the dependent quantity in which the parameters $k_o$ and $E$ will be estimated.

This model is then defined in the form of a python subroutine (**def**) and represented by:

In [3]:
def Model (param, x, args):

    ko, E = param[0], param[1]
    time, T = x[:,0], x[:,1]

    return exp(-(ko*10**17)*time*exp(-E/T))

## Class initialization

The first step to perform the estimation is to configure the class **EstimacaoNaoLinear** through the inclusion of basic information.:

* The model,
* List of symbols for quantity *(x)*; 
* List of symbols for quantity *(y)*; 
* The symbols of the parameters *(param)*;
* The project name, the folder's name where the results will be saved.

In [4]:
Estime = EstimacaoNaoLinear(Model, symbols_x=[r't','Tao'], symbols_y=[r'y'],
                            symbols_param=['ko','E'], Folder='Example1')

## Data inclusion

The experimental data provided in Table 1 of the dependent quantity (y) and the independent quantities (t and T), are presented below in the form of lists:

In [5]:
#Time
time = [120.0,60.0,60.0,120.0,120.0,60.0,60.0,30.0,15.0,60.0,
45.1,90.0,150.0,60.0,60.0,60.0,30.0,90.0,150.0,90.4,120.0,
60.0,60.0,60.0,60.0,60.0,60.0,30.0,45.1,30.0,30.0,45.0,15.0,30.0,90.0,25.0,
60.1,60.0,30.0,30.0,60.0]

#Temperature
temperature = [600.0,600.0,612.0,612.0,612.0,612.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,631.0,631.0,631.0,631.0,631.0,639.0,639.0,
639.0,639.0,639.0,639.0,639.0,639.0,639.0]

#Reagent fraction 
y = [0.9,0.949,0.886,0.785,0.791,0.890,0.787,0.877,0.938,
0.782,0.827,0.696,0.582,0.795,0.800,0.790,0.883,0.712,0.576,0.715,0.673,
0.802,0.802,0.804,0.794,0.804,0.799,0.764,0.688,0.717,0.802,0.695,0.808,
0.655,0.309,0.689,0.437,0.425,0.638,.659,0.449]

The MT_PEU **needs** the **uncertainties of the experimental data** (ux1, ux2, uy1) to be informed. 

In [6]:
uy = [0.02]*41
uxtime = [0.02]*41
uxtemperature = [0.02]*41

 **Entering the experimental data in MT_PEU:**

The *setDados* method is used to include the data for dependent and independent quantities. Syntax:

* Quantity identification, whether it is independent or dependent: 0 or 1 (respectively)
* The experimental data and their uncertainties must be entered in sequence in the form of tuples.

In [7]:
Estime.setDados(data={0:[(time,uxtime),(temperature,uxtemperature)],1:[(y,uy)]})

Defines that the experimental data previously entered will be used as a data set for which parameters will be estimated:

## Optimization

In this example, the user can choose the algorithm to be used in the optimization. 
Available: 'ipopt', 'bonmin', 'sqpmethod'. 
If the user does not choose any optimization method, the default algorithm will be used: ipopt, with initial estimative equal to [0.5, 25000.000].

In [8]:
Estime.optimize(initial_estimative= [0.5, 25000.000])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



## Parameters uncertainty


In this example, you can choose the method used to assess uncertainty. Available: 2InvHessiana, General, SensitivityModel. By definition, filling in the probability region is 'True' and the uncertainty method is General, if necessary, these options can be changed.

If the user **does not** have an interest in **evaluating the uncertainty of the parameters**, just **do not** perform **Estime.parametersUncertainty**.

In [9]:
Estime.parametersUncertainty()

## Prediction and residual analysis

The prediction method evaluates the dependent quantity based on the estimated parameters, the model informed by the user, and data of the dependent quantities. In case validation data are informed (through the setConjunto method, defining "dataType = predicao") these will be used to perform the prediction. Also, this method evaluates the covariance matrix of the prediction, in case the parametersUncertainty method has been performed.
 
Through residual analysis, it is possible to obtain indicators of estimation quality.  Aspects such as mean, homoscedasticity (which allows us to infer possible dependency and/or tendency relationships between the variables), normality, $R^2$ and autocorrelation are evaluated. The optimal point of the objective function is also evaluated. The residues analysis is performed primarily with the validation data.

In [10]:
Estime.prediction()
Estime.residualAnalysis()

C:\Users\artur\anaconda3\lib\site-packages\statsmodels\stats\diagnostic.py:559: FutureWarning: The value returned will change to a single DataFrame after 0.12 is released.  Set return_df to True to use to return a DataFrame now.  Set return_df to False to silence this warning.
  warnings.warn(msg, FutureWarning)


## Plots and reports

At this stage, the results obtained by the program are exported: reports and graphs. 
The graphs are generated according to the steps that have been performed. In the reports, information about the statistical tests, objective function value, covariance matrix of the parameters, optimization status, among others, are printed.

In [ ]:
Estime.plots()
Estime.reports()

## References: 

[1] SRINIVASAN, R.; LEVI, A. A.. Kinetics of the Thermal Isomerization of Bicyclo [2.1.1 ]hexane. Journal Of The American Chemical Society, [s.l.], v. 85, n. 21, p.3363-3365, 5 nov. 1963. American Chemical Society (ACS)

[2] SCHWAAB, M.M.;PINTO, J.C. Análise de Dados Experimentais I: Fundamentos da Estátistica e Estimação de Parâmetros. 
Rio de Janeiro: e-papers, 2007.

INMETRO.: Avaliação de dados de medição — Guia para a expressão de incerteza de medição. Rio de Janeiro: Jcgm, 2008.